In [1]:
pip install richdem

  Using cached richdem-0.3.4-cp36-cp36m-manylinux1_x86_64.whl (4.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pwd

'/home/jovyan/dev/dea-notebooks/Gabrielle_dev/LCCS_wet_veg'

In [3]:
import richdem as rd
import pyproj
from odc.algo import xr_reproject
import datacube
import numpy as np
import sys
import xarray as xr

sys.path.append('Scripts')
from dea_bandindices import calculate_indices
# from deafrica_temporal_statistics import xr_phenology, temporal_statistics
from datacube.utils.geometry import assign_crs

def xr_terrain(da, attribute=None):
    """
    Using the richdem package, calculates terrain attributes
    on a DEM stored in memory as an xarray.DataArray 
    
    Params
    -------
    da : xr.DataArray
    attribute : str
        One of the terrain attributes that richdem.TerrainAttribute()
        has implemented. e.g. 'slope_riserun', 'slope_percentage', 'aspect'.
        See all option here:  
        https://richdem.readthedocs.io/en/latest/python_api.html#richdem.TerrainAttribute
        
    """
    #remove time if its there
    da = da.squeeze()
    #convert to richdem array
    rda = rd.rdarray(da.data, no_data=da.attrs['nodata'])
    #add projection and geotransform
    rda.projection=pyproj.crs.CRS(da.attrs['crs']).to_wkt()
    rda.geotransform = da.geobox.affine.to_gdal()
    #calulate attribute
    attrs = rd.TerrainAttribute(rda, attrib=attribute)

    #return as xarray DataArray
    return xr.DataArray(attrs,
                        attrs=da.attrs,
                        coords={'x':da.x, 'y':da.y},
                        dims=['y', 'x'])


def crop_features(ds):
    dc = datacube.Datacube(app='training')
    data = calculate_indices(ds,
                             index=['NDVI'],
                             drop=True,
                             collection='s2')
    
    #temporal stats
#     ts = temporal_statistics(data.NDVI,
#                        stats=['f_mean', 'abs_change',
#                               'complexity','central_diff'])
    ts = xr_phenology(data.NDVI, 
                      stats=['Trough','vSOS', 'vPOS','AOS','ROG','ROS'],
                      complete='fast_completion')

    #rainfall climatology
    chirps = assign_crs(xr.open_rasterio('data/CHIRPS/CHPclim_sum.nc'),  crs='epsg:4326')
    chirps = xr_reproject(chirps,ds.geobox,"mode")
    chirps = chirps.to_dataset(name='chirps')
    
    #slope
    slope = dc.load(product='srtm', like=ds.geobox).squeeze()
    slope = slope.elevation
    slope = xr_terrain(slope, 'slope_riserun')
    slope = slope.to_dataset(name='slope')
    
    #Surface reflectance results
    sr = ds.median('time')
    result = xr.merge([ts, sr, chirps,slope], compat='override')
    result = assign_crs(result, crs=ds.geobox.crs)

    return result.squeeze()

In [4]:
import datacube

dc = datacube.Datacube(app='tcw')

In [6]:
# list(dc.list_products()['name'])

In [7]:
tile = '15_-40'

In [8]:
# Get tile bounds
def get_tile_bounds(tile):
    x, y = tile.split('_')
    minx = int(x) * 100000
    miny = int(y) * 100000
    maxx = minx + 100000
    maxy = miny + 100000
    return {'minx': minx, 'miny': miny, 'maxx': maxx, 'maxy': maxy}
        
          

In [9]:
bounds = get_tile_bounds(tile)
print(bounds)

{'minx': 1500000, 'miny': -4000000, 'maxx': 1600000, 'maxy': -3900000}


In [10]:
# Load geomedian data
ds_gm = dc.load(product="ls8_nbart_geomedian_annual",
             x=(bounds['minx'],bounds['maxx']),
             y=(bounds['miny'],bounds['maxy']),
             time=("2015-01-01", "2015-12-31"),
             crs='EPSG:3577',
             output_crs = 'EPSG:3577',
             #min_gooddata = 0.5,
             #bands = ['nbart_red','nbart_green','nbart_blue','nbart_nir','nbart_swir_1','nbart_swir_2'],
             resolution=(-100, 100))

print(ds_gm)

<xarray.Dataset>
Dimensions:      (time: 1, x: 1000, y: 1000)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 -3.9e+06 -3.9e+06 -3.9e+06 ... -4e+06 -4e+06 -4e+06
  * x            (x) float64 1.5e+06 1.5e+06 1.5e+06 ... 1.6e+06 1.6e+06 1.6e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 420 405 384 399 396 ... 200 204 157 155 160
    green        (time, y, x) int16 702 679 671 706 716 ... 327 320 246 243 261
    red          (time, y, x) int16 788 751 696 720 718 ... 310 309 241 231 227
    nir          (time, y, x) int16 2614 2440 2444 2694 ... 2298 2305 2229 2156
    swir1        (time, y, x) int16 2452 2341 2254 2464 ... 1166 846 874 1020
    swir2        (time, y, x) int16 1484 1418 1326 1442 1448 ... 555 353 369 461
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


In [13]:
dem = dc.load(product='srtm_dem1sv1_0', 
                x=(bounds['minx'],bounds['maxx']),
                y=(bounds['miny'],bounds['maxy']),
#                 crs='EPSG:3577',
#                 output_crs = 'EPSG:3577',
                resolution=(-25, 25)).squeeze()
slope = dem.dem #no elevation
slope = xr_terrain(slope, 'slope_riserun')
slope = slope.to_dataset(name='slope')

ValueError: No products match search terms: {'lat': Range(begin=-4000000.0, end=-3900000.0), 'lon': Range(begin=1500000.0, end=1600000.0), 'product': 'srtm_dem1sv1_0'}